### RankForecasting-shortterm-paper

test 1k pitmodel

basedon :stint-test-strategy-shortterm
    
+ comparing models: naive, deepar, oracle, pitmodel

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os,sys
import mxnet as mx
from mxnet import gluon
import pickle
import json
import random
import inspect
from scipy import stats
from sklearn.metrics import mean_squared_error
from gluonts.dataset.common import ListDataset
from gluonts.dataset.util import to_pandas
from pathlib import Path
from gluonts.model.deepar import DeepAREstimator
from gluonts.model.deep_factor import DeepFactorEstimator
from gluonts.model.deepstate import DeepStateEstimator
from gluonts.trainer import Trainer
from gluonts.model.simple_feedforward import SimpleFeedForwardEstimator
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator, MultivariateEvaluator
from gluonts.distribution.multivariate_gaussian import MultivariateGaussianOutput
from gluonts.model.predictor import Predictor
from gluonts.model.prophet import ProphetPredictor
from gluonts.model.r_forecast import RForecastPredictor
from indycar.model.NaivePredictor import NaivePredictor
from indycar.model.ZeroPredictor import ZeroPredictor
from pandas.plotting import register_matplotlib_converters
from gluonts.model.forecast import SampleForecast
register_matplotlib_converters()

#from indycar.model.stint_predictor_fastrun import *
#import indycar.model.stint_simulator_shortterm_pitmodel as stint
import indycar.model.stint_simulator_paper as stint

INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU


In [2]:
def simulation(datasetid, testevent, taskid, runts, expid, predictionlen, 
               datamode, loopcnt, featuremode = stint.FEATURE_STATUS,
              pitmodel = 0, model = 'oracle', inlapmode=0, exproot='remote'):
    #
    # configurataion
    #
    # model path:  <_dataset_id>/<_task_id>-<trainid>/
    #_dataset_id = 'indy2013-2018-nocarid'
    
    stint._inlap_status = inlapmode
    
    stint.init(pitmodel)
    stint._dataset_id = datasetid
    stint._test_event = testevent
    #_test_event = 'Indy500-2019'

    stint._feature_mode = featuremode
    stint._context_ratio = 0.

    stint._task_id = taskid  # rank,laptime, the trained model's task
    stint._run_ts = runts   #COL_LAPTIME,COL_RANK
    stint._exp_id=expid  #rank, laptime, laptim2rank, timediff2rank... 

    stint._train_len = 40
    
    stint._use_mean = True
        
    #predictor = stint.load_model(predictionlen, model,trainid='2018',exproot=exproot)
    predictor = stint.load_model(predictionlen, model,trainid='indy500',exproot=exproot)

    ret2 = {}
    for i in range(loopcnt):
        #df, full_samples, full_tss
        ret2[i] = stint.run_simulation_shortterm(predictor, predictionlen, stint.freq, datamode=datamode)

    acc = []
    for i in ret2.keys():
        df = ret2[i][0]
        _x = stint.get_evalret_shortterm(df)
        acc.append(_x)

    b = np.array(acc)
    print(np.mean(b, axis=0))
    
    #save keys
    #stint._pitmodel.save_keys('pitmodel-keys.pickle')
    
    return b, ret2

In [3]:
def test_pitmodel(modelfile, testevent, runts, loopcnt=10, model='oracle', 
                  inlapmode=0,featuremode = stint.FEATURE_PITAGE,
                 predictionlen=2, exproot='remote'):
    #pitmodel-m60-mlp-e500-l10-10-5-student-d0.1.pickle
    
    lapmode = {0:'noinlap', 1:'inlap',2:'outlap'}
    
    #dataset = 'indy2013-2018%s-nocarid-context40'%('' if inlapmode==0 else '-inlapstatus')
    #dataset = 'indy2013-2018-%s-%s-nocarid-context40'%(
    dataset = 'indy2013-2018-%s-%s'%(
                lapmode[inlapmode],
                'pitage' if featuremode==stint.FEATURE_PITAGE else 'nopitage'
                )
    #try:
    if True:
        if runts == 'RANK':
            acc, ret = simulation(dataset, testevent, 
                        'rank',stint.COL_RANK,'rank',
                       predictionlen, stint.MODE_ORACLE_LAPONLY,loopcnt, 
                          pitmodel=modelfile, model=model, inlapmode=inlapmode,featuremode =featuremode,
                                 exproot=exproot)        
        else:
            acc, ret = simulation(dataset, testevent, 
                            'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                           predictionlen, stint.MODE_ORACLE_LAPONLY,loopcnt, 
                              pitmodel=modelfile, model=model, inlapmode=inlapmode,featuremode =featuremode,
                                 exproot=exproot)
    #except:
    #    acc, ret = [],[]
        
    #print(np.mean(acc, axis=0))
    return acc, ret

In [4]:
def get_samples(startlap, carno=12):
    samples = ret['pitmodel-RANK-2018-inlap-nopitage'].keys()
    retdf = []
    for id in samples:
        df = ret['pitmodel-RANK-2018-inlap-nopitage'][id]
        rec = df[(df['carno']==carno) & (df['startlap']==startlap)]
        retdf.append(rec)
        
    dfout = pd.concat(retdf)
    return dfout



def get_alldf(year=2018, model='pitmodel'):
    

    dfx = ret[f'{model}-RANK-{year}-inlap-nopitage']
    #dfx = ret[f'{model}-TIMEDIFF-{year}-noinlap-nopitage']
    
    samples = dfx.keys()
    retdfs = []
    for id in samples:
        df = dfx[id][0]
        retdfs.append(df)
        
    if len(retdfs) > 1:
        dfout = pd.concat(retdfs)
    else:
        dfout = retdfs[0]
        
    return dfout
    
def get_alldf_mode(year=2018,mode=0,model='pitmodel'):
    """
    mode: 
        0; mode
        1; mean
        2; median
    """
    dfall = get_alldf(year=year, model=model)
    
    cars = set(dfall.carno.values)
    startlaps = {}
    for car in cars:
        startlaps[car] = set(dfall[dfall['carno']==car].startlap.values)
        
    retdf = []
    for car in cars:
        for startlap in startlaps[car]:
            dfrec = dfall[(dfall['carno']==car) & (dfall['startlap']==startlap)]
            
            #get mode
            if mode == 0:
                pred_endrank = stats.mode(dfrec.pred_endrank.values).mode[0]
                #pred_endlap =  stats.mode(dfrec.pred_endlap.values).mode[0]
            elif mode == 1:
                #use mean
                pred_endrank = np.mean(dfrec.pred_endrank.values)
                #pred_endlap =  np.mean(dfrec.pred_endlap.values)
            elif mode == 2:
                #use mean
                pred_endrank = np.median(dfrec.pred_endrank.values)
                #pred_endlap =  np.median(dfrec.pred_endlap.values)
            
            firstrec = dfrec.to_numpy()[0,:]
            firstrec[6] = pred_endrank
            firstrec[7] = pred_endrank - firstrec[2]
            if firstrec[7] == 0:
                firstrec[8] = 0
            elif firstrec[7] > 0:
                firstrec[8] = 1
            else:
                firstrec[8] = -1
                
            #endlap, pred_endlap
            
        
            retdf.append(firstrec)
        
    #dfout = pd.concat(retdf)
    dfout = pd.DataFrame(retdf, columns =['carno', 'startlap', 'startrank',    
                                         'endrank', 'diff', 'sign',
                                         'pred_endrank', 'pred_diff', 'pred_sign',
                                         #'endlap','pred_endlap'
                                        ])
    print('df size:', len(dfout))
    return dfout

In [5]:
def get_prisk_ex(year=2018, model='pitmodel'):
    dfx = ret[f'{model}-RANK-{year}-inlap-nopitage']
    allsamples, alltss = stint.get_allsamples_ex(dfx)
    _, prisk_vals =stint.prisk_direct_bysamples(allsamples, alltss)
    
    return prisk_vals

def get_prisk(ret, mid):
    dfx = ret[mid]
    allsamples, alltss = stint.get_allsamples_ex(dfx)
    _, prisk_vals =stint.prisk_direct_bysamples(allsamples, alltss)
    
    return prisk_vals

def get_best(pitmodel=True, verbose=False):
    best_acc = {2018:[0, ''], 2019:[0,'']}
    best_mae = {2018:[100, ''], 2019:[100,'']}
    for model in ret.keys():
        if model.find('pitmodel')==0:
            if len(ret[model]) > 0:
                oracle = model.replace('pitmodel','oracle')
                if verbose:
                    print('model:',(oracle))
                acc_oracle = np.mean(acc[oracle], axis=0)
                if verbose:
                    print(acc_oracle)

                if verbose:
                    print('model:',(model))
                acc_model = np.mean(acc[model], axis=0)
                if verbose:
                    print(acc_model)

                if model.find('2018') > 0:
                    year = 2018
                else:
                    year = 2019
                    
                if pitmodel:
                    test_model = acc_model
                    modelname = model
                else:
                    test_model = acc_oracle
                    modelname = oracle
                    
                #mae
                if test_model[0, 0] > best_acc[year][0]:
                    best_acc[year][0] = test_model[0, 0]
                    best_acc[year][1] = modelname

                if test_model[0, 1] < best_mae[year][0]:
                    best_mae[year][0] = test_model[0, 1]
                    best_mae[year][1] = modelname
                
    #print('mae',best_mae)
    #print('acc',best_acc)                
    return best_acc, best_mae

In [6]:
import pickle
def save_result(dfs, datafile):
    with open(datafile, 'wb') as f:
        #pack [global_carids, laptime_data]
        savedata = [dfs]
        #savedata = [freq, train_set, test_set]
        # Pickle the 'data' dictionary using the highest protocol available.
        pickle.dump(savedata, f, pickle.HIGHEST_PROTOCOL)

### simulation with 100 runs

In [7]:
#_exproot = 'remote'
_exproot = 'ranknet-train'
_usemean = True

In [8]:
fmodestr = {stint.FEATURE_PITAGE:'pitage',stint.FEATURE_STATUS:'nopitage'}
lapstr = {0:'noinlap',1:'inlap',2:'outlap'}
usemeanstr = {True:'mean',False:'median'}
acc, ret, pret = {}, {}, {}

In [9]:
#
#  pitmodel rank-inlap-nopitage
# 
for plen in [2,4,6,8]:
    
    _predictionlen = plen
    
    for year in ['2018','2019']:
        lapmode = 1
        fmode = stint.FEATURE_STATUS
        runts = 'RANK'
        mid = 'pitmodel-%s-%s-%s-%s'%(runts, year, lapstr[lapmode], fmodestr[fmode])
        acc[mid], ret[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1-1k.pickle', 
                                        'Indy500-%s'%year, runts, loopcnt=100,inlapmode=lapmode,featuremode = fmode,
                                          predictionlen=_predictionlen, exproot=_exproot)

    #
    # oracle rank-inlap-nopitage
    #
    for year in ['2018','2019']:
        lapmode = 1
        fmode = stint.FEATURE_STATUS
        runts = 'RANK'
        mid = 'oracle-%s-%s-%s-%s'%(runts, year, lapstr[lapmode], fmodestr[fmode])
        acc[mid], ret[mid] = test_pitmodel('oracle', 'Indy500-%s'%year, runts, loopcnt=10,inlapmode=lapmode,
                                           featuremode = fmode,predictionlen=_predictionlen, exproot=_exproot)

    ### get prisk
    for mid in ret:
        prisk_vals = get_prisk(ret, mid)
        pret[mid] = prisk_vals
        print('mid:',mid, 'prisk:',prisk_vals)

    dfs={}
    for model in ['pitmodel','oracle']:
        #for model in ['pitmodel']:
        for year in ['2018','2019']:
            for mode in [0,1,2]:

                df = get_alldf_mode(year=year,mode=mode,model=model)
                name = '%s_%s'%(model, 'mean' if mode==1 else ('mode' if mode==0 else 'median'))
                if year not in dfs:
                    dfs[year] = {}
                dfs[year][name] = df

    _trim = 0
    _include_final = True
    _include_stintlen = True
    #_include_stintlen = False
    include_str = '1' if _include_final else '0'
    stint_str = '1' if _include_stintlen else ''            
    outfile=f'shortterm-dfout-ranknet-indy500-rank-inlap-nopitage-20182019-{usemeanstr[_usemean]}-t{_predictionlen}-alldata.pickle'
    save_result([dfs,acc,ret,pret], outfile)    

    ### parameter tuning
    acc, ret, pret = {}, {}, {}

    for runts in ['TIMEDIFF','RANK']:
        for fmode in [stint.FEATURE_PITAGE, stint.FEATURE_STATUS]:
            for year in ['2018','2019']:
                for lapmode in [0,1,2]:
                    mid = 'oracle-%s-%s-%s-%s'%(runts, year, lapstr[lapmode], fmodestr[fmode])
                    acc[mid], ret[mid] = test_pitmodel('oracle', 'Indy500-%s'%year, runts, loopcnt=1,
                                                       inlapmode=lapmode,featuremode = fmode,
                                                      predictionlen=_predictionlen, exproot=_exproot)

    for runts in ['TIMEDIFF','RANK']:
        for fmode in [stint.FEATURE_PITAGE, stint.FEATURE_STATUS]:
            for year in ['2018','2019']:
                for lapmode in [0,1,2]:
                    mid = 'pitmodel-%s-%s-%s-%s'%(runts, year, lapstr[lapmode], fmodestr[fmode])
                    acc[mid], ret[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1-1k.pickle', 
                                        'Indy500-%s'%year, runts, loopcnt=3,inlapmode=lapmode,
                                                       featuremode = fmode,predictionlen=_predictionlen, exproot=_exproot)

    ### get prisk
    for mid in ret:
        prisk_vals = get_prisk(ret, mid)
        pret[mid] = prisk_vals
        print('mid:',mid, 'prisk:',prisk_vals)

    
    print(f'best model (plen={_predictionlen}):')
    print(get_best(pitmodel=True, verbose=False))
    
    print(get_best(pitmodel=False, verbose=False))
    
    outfile=f'shortterm-dfout-ranknet-indy500-rank-inlap-nopitage-20182019-{usemeanstr[_usemean]}-t{_predictionlen}-tuning.pickle'
    save_result([[],acc,ret,pret], outfile)

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_paper.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_paper.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p1.pickle with 7 races, 58 cars
init model:pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1-1k.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model


model: acc={0.75}, mae={1.33}, rmse={3.13},r2={0.86}, {188}
            naive: acc={0.72}, mae={1.34}, rmse={3.25},r2={0.85}
model: acc={0.74}, mae={1.34}, rmse={3.15},r2={0.86}, {188}
            naive: acc={0.72}, mae={1.34}, rmse={3.25},r2={0.85}
model: acc={0.75}, mae={1.32}, rmse={3.14},r2={0.86}, {188}
            naive: acc={0.72}, mae={1.34}, rmse={3.25},r2={0.85}
model: acc={0.77}, mae={1.32}, rmse={3.13},r2={0.86}, {188}
            naive: acc={0.72}, mae={1.34}, rmse={3.25},r2={0.85}
model: acc={0.74}, mae={1.34}, rmse={3.15},r2={0.86}, {188}
            naive: acc={0.72}, mae={1.34}, rmse={3.25},r2={0.85}
model: acc={0.78}, mae={1.32}, rmse={3.08},r2={0.87}, {188}
            naive: acc={0.72}, mae={1.34}, rmse={3.25},r2={0.85}
model: acc={0.74}, mae={1.33}, rmse={3.15},r2={0.86}, {188}
            naive: acc={0.72}, mae={1.34}, rmse={3.25},r2={0.85}
model: acc={0.73}, mae={1.32}, rmse={3.12},r2={0.86}, {188}
            naive: acc={0.72}, mae={1.34}, rmse={3.25},r2={0.85}


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_paper.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_paper.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p1.pickle with 7 races, 58 cars
init model:pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1-1k.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=

...

init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p1.pickle with 7 races, 58 cars
init model:pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1-1k.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=8
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
model: acc={0.45}, mae={2.47}, rmse={4.51},r2={0.73}, {182}
            naive: acc={0.39}, mae={2.69}, rmse={5.00},r2={0.67}
model: acc={0.47}, mae={2.49}, rmse={4.52},r2={0.73}, {182}
            naive: acc={0.39}, mae={2.69}, rmse={5.00},r2={0.67}
model: acc={0.48}, mae={2.49}, rmse={4.54},r2={0.73}, {182}
            naive: acc={0.39}, mae={2.69}, rmse={5.00},r2={0.67}
[[0.46520147 2.48127031 4.52354967 0.7315683 ]
 [0.39010989 2.6860401  4.99957695 0.67210394]]


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_paper.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_paper.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p2.pickle with 7 races, 58 cars
init model:pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1-1k.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=8
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
model: acc={0.49}, mae={2.34}, rmse={4.37},r2={0.75}, {182}
            naive: acc={0.39}, mae={2.69}, rmse={5.00},r2={0.67}
model: acc={0.48}, mae={2.34}, rmse={4.39},r2={0.75}, {182}
            naive: acc={0.39}, mae={2.69}, rmse={5.00},r2={0.67}
model: acc={0.47}, mae={2.32}, rmse={4.36},r2={0.75}, {182}
            naive: acc={0.39}, mae={2.69}, rmse={5.00},r2={0.67}
[[0.48168498 2.33363987 4.37297731 0.7491427 ]
 [0.39010989 2.6860401  4.99957695 0.67210394]]
sacmplecnt: 100 lapcnt: 200 runcnt: 1
dict_values([0.16650760969366468, 0.1665076096936646, 0.1665076096936646])
mid: oracle-TIMEDIFF-2018-noinlap-pitage prisk: [0.16650

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_paper.py:3535: RuntimeWarning: Mean of empty slice
  forecast_mean = np.nanmean(forecast[carno], axis=0)
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_paper.py:3282: RuntimeWarning: invalid value encountered in less_equal
  * ((target <= quantile_forecast) - q)


dict_values([0.13613104181171903, 0.13613104181171898, 0.13613104181171903])
mid: oracle-RANK-2019-outlap-pitage prisk: [0.13613104 0.13613104 0.13613104]
sacmplecnt: 100 lapcnt: 200 runcnt: 1
dict_values([0.2030313569461061, 0.20303135694610608, 0.20303135694610616])
mid: oracle-RANK-2018-noinlap-nopitage prisk: [0.20303136 0.20303136 0.20303136]
sacmplecnt: 100 lapcnt: 200 runcnt: 1
dict_values([0.2119281600400657, 0.21192816004006565, 0.21192816004006568])
mid: oracle-RANK-2018-inlap-nopitage prisk: [0.21192816 0.21192816 0.21192816]
sacmplecnt: 100 lapcnt: 200 runcnt: 1
dict_values([0.20260037283325452, 0.20260037283325447, 0.2026003728332545])
mid: oracle-RANK-2018-outlap-nopitage prisk: [0.20260037 0.20260037 0.20260037]
sacmplecnt: 100 lapcnt: 200 runcnt: 1
dict_values([0.14505820416040652, 0.14505820416040652, 0.14505820416040652])
mid: oracle-RANK-2019-noinlap-nopitage prisk: [0.1450582 0.1450582 0.1450582]
sacmplecnt: 100 lapcnt: 200 runcnt: 1
dict_values([0.1501317991376257,